# COGS 118B Project





In [61]:
import csv
import pandas as pd
import numpy as np
import statsmodels.formula.api as sm
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
import numpy as np
import matplotlib.pyplot as plt
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.datasets import load_digits
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score
from sklearn.neural_network import MLPClassifier
import warnings
warnings.filterwarnings("ignore")

#name the columns
colnames=['age', 'workclass',  'fnlwgt', 'education', 
          'education-num', 'marital-status', 'occupation', 
          'relationship', 'race', 'sex', 'capital-gain', 
          'capital-loss', 'hours-per-week', 'native-country', 'income']

# read data in df
df = pd.read_csv('adult.data', names=colnames, header=None)


In [2]:
# display df header
data_top = df.head()   
data_top

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


### Drop null values

In [3]:
df.shape #before dropping nulls

(32561, 15)

In [4]:
for i in colnames:   #ignores all null values
    df = df[(df[i] != ' ?')]

In [5]:
df.shape #after dropping nulls

(30162, 15)

### Keep income only from Americans since it represents most of the data and remove trailing white space

In [6]:
df = df[(df['native-country'] == ' United-States')]
df = df.reset_index()

In [7]:
df = df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

### One hot encode

In [8]:
new_df = df[['age','fnlwgt','education-num','capital-gain',
             'capital-loss','hours-per-week']]
one_hot_cols = ['workclass', 'marital-status',
                'occupation','relationship','race']
for i in one_hot_cols:
    new_df = new_df.join(pd.get_dummies(df[i],prefix = i))
    
new_df['income'] = pd.DataFrame((df['income'] == '>50K').astype(int))

In [9]:
for i in new_df.columns: #print new columns
    print(i)

age
fnlwgt
education-num
capital-gain
capital-loss
hours-per-week
workclass_Federal-gov
workclass_Local-gov
workclass_Private
workclass_Self-emp-inc
workclass_Self-emp-not-inc
workclass_State-gov
workclass_Without-pay
marital-status_Divorced
marital-status_Married-AF-spouse
marital-status_Married-civ-spouse
marital-status_Married-spouse-absent
marital-status_Never-married
marital-status_Separated
marital-status_Widowed
occupation_Adm-clerical
occupation_Armed-Forces
occupation_Craft-repair
occupation_Exec-managerial
occupation_Farming-fishing
occupation_Handlers-cleaners
occupation_Machine-op-inspct
occupation_Other-service
occupation_Priv-house-serv
occupation_Prof-specialty
occupation_Protective-serv
occupation_Sales
occupation_Tech-support
occupation_Transport-moving
relationship_Husband
relationship_Not-in-family
relationship_Other-relative
relationship_Own-child
relationship_Unmarried
relationship_Wife
race_Amer-Indian-Eskimo
race_Asian-Pac-Islander
race_Black
race_Other
race_Whit

In [10]:
new_df.head(10)

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week,workclass_Federal-gov,workclass_Local-gov,workclass_Private,workclass_Self-emp-inc,...,relationship_Other-relative,relationship_Own-child,relationship_Unmarried,relationship_Wife,race_Amer-Indian-Eskimo,race_Asian-Pac-Islander,race_Black,race_Other,race_White,income
0,39,77516,13,2174,0,40,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,50,83311,13,0,0,13,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,38,215646,9,0,0,40,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,53,234721,7,0,0,40,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0
4,37,284582,14,0,0,40,0,0,1,0,...,0,0,0,1,0,0,0,0,1,0
5,52,209642,9,0,0,45,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
6,31,45781,14,14084,0,50,0,0,1,0,...,0,0,0,0,0,0,0,0,1,1
7,42,159449,13,5178,0,40,0,0,1,0,...,0,0,0,0,0,0,0,0,1,1
8,37,280464,10,0,0,80,0,0,1,0,...,0,0,0,0,0,0,1,0,0,1
9,23,122272,13,0,0,30,0,0,1,0,...,0,1,0,0,0,0,0,0,1,0


In [11]:
new_df.shape   #shape after all cleaning

(27504, 46)

In [15]:
new_df = new_df.reset_index()

In [16]:
new_df.columns

Index(['index', 'age', 'fnlwgt', 'education-num', 'capital-gain',
       'capital-loss', 'hours-per-week', 'workclass_Federal-gov',
       'workclass_Local-gov', 'workclass_Private', 'workclass_Self-emp-inc',
       'workclass_Self-emp-not-inc', 'workclass_State-gov',
       'workclass_Without-pay', 'marital-status_Divorced',
       'marital-status_Married-AF-spouse', 'marital-status_Married-civ-spouse',
       'marital-status_Married-spouse-absent', 'marital-status_Never-married',
       'marital-status_Separated', 'marital-status_Widowed',
       'occupation_Adm-clerical', 'occupation_Armed-Forces',
       'occupation_Craft-repair', 'occupation_Exec-managerial',
       'occupation_Farming-fishing', 'occupation_Handlers-cleaners',
       'occupation_Machine-op-inspct', 'occupation_Other-service',
       'occupation_Priv-house-serv', 'occupation_Prof-specialty',
       'occupation_Protective-serv', 'occupation_Sales',
       'occupation_Tech-support', 'occupation_Transport-moving',
   

### Get indexes for each individual feature

In [18]:
ranges = [[1,2],[2,3],[3,4],[4,5],[5,6],[6,7],[7,14],[14,21],[21,35],[35,41],[41,46]]
columns = ['Age','Final Weight', 'Education','Capital Gain','Capital Loss','Hours Per Week',
           'Workclass','Marital Status','Occupation','Relationship','Race']

### Calculate accuracy per each feature

In [62]:
Y = np.array(new_df.iloc[:,46:47])
num_training = int(0.8*X.shape[0])
for i in range(len(ranges)):
    X = sc.fit_transform(new_df.iloc[:,ranges[i][0]:ranges[i][1]])
    
    X_train = X[:num_training]
    X_test = X[num_training:]
    Y_train = Y[:num_training]
    Y_test = Y[num_training:]
    
    clf = SVC(kernel = 'linear',gamma ='auto')
    clf.fit(X_train, Y_train)
    scores = cross_val_score(clf, X_test, Y_test, cv=10)
    print('Using' ,columns[i],'as a predictor for SVM we get accuracy of: ',max(scores))
    
    clf = MLPClassifier(solver='lbfgs', alpha=10,hidden_layer_sizes=(5, 2))
    clf.fit(X_train,Y_train)
    print('Using' ,columns[i],'as a predictor for a Neural Network we get accuracy of: ',clf.score(X_test,Y_test))
    print()

X = np.array(new_df.iloc[:,1:46])
Y = np.array(new_df.iloc[:,46:47])
sc = StandardScaler()
X = sc.fit_transform(X)

num_training = int(0.8*X.shape[0])
X_train = X[:num_training]
X_test = X[num_training:]
Y_train = Y[:num_training]
Y_test = Y[num_training:]

clf = SVC(kernel = 'linear',gamma ='auto')
clf.fit(X_train, Y_train)
scores = cross_val_score(clf, X_test, Y_test, cv=5)
print('Using all columns as predictors for SVM we get accuracy of: ',max(scores))
clf = MLPClassifier(solver='lbfgs', alpha=10,hidden_layer_sizes=(5, 2))
clf.fit(X_train,Y_train)
print('Using all columns as predictors for a Neural Network we get accuracy of: ',clf.score(X_test,Y_test))

Using Age as a predictor for SVM we get accuracy of:  0.74
Using Age as a predictor for a Neural Network we get accuracy of:  0.7398654790038175

Using Final Weight as a predictor for SVM we get accuracy of:  0.74
Using Final Weight as a predictor for a Neural Network we get accuracy of:  0.7398654790038175

Using Education as a predictor for SVM we get accuracy of:  0.74
Using Education as a predictor for a Neural Network we get accuracy of:  0.7398654790038175

Using Capital Gain as a predictor for SVM we get accuracy of:  0.7963636363636364
Using Capital Gain as a predictor for a Neural Network we get accuracy of:  0.7398654790038175

Using Capital Loss as a predictor for SVM we get accuracy of:  0.7545454545454545
Using Capital Loss as a predictor for a Neural Network we get accuracy of:  0.7567714960916198

Using Hours Per Week as a predictor for SVM we get accuracy of:  0.74
Using Hours Per Week as a predictor for a Neural Network we get accuracy of:  0.7398654790038175

Using Wo